In [88]:
'''
Install this features first in odl:
feature:install odl-mdsal-clustering
feature:install odl-restconf
feature:install odl-mdsal-apidocs
feature:install odl-vtn-manager
'''

'\nInstall this features first in odl:\nfeature:install odl-mdsal-clustering\nfeature:install odl-restconf\nfeature:install odl-mdsal-apidocs\nfeature:install odl-vtn-manager\n'

In [1]:
from vtnManager.vtnManager import ODLClient
odl = ODLClient("http://localhost:8181")

In [2]:
odl.set_flow_cond("cond_prueba", "100", "10.0.0.1", "10.0.0.2")

{'input': {'operation': 'SET', 'present': 'false', 'name': 'cond_prueba', 'vtn-flow-match': [{'vtn-ether-match': {}, 'vtn-inet-match': {'source-network': '10.0.0.1/32', 'protocol': 1, 'destination-network': '10.0.0.2/32'}, 'index': '100'}]}}


{'output': {'status': 'CHANGED'}}

In [2]:
odl.set_path_map("cond_1", "1", "1", "triangle")

{'output': {'set-path-map-result': [{'index': 1, 'status': 'CREATED'}]}}

In [20]:
odl.set_path_map("cond4", "4", "1", "square")

{'output': {'set-path-map-result': [{'index': 4}]}}

In [2]:
odl.set_path_policy({"s3-eth1": "100", "s4-eth2": "1000"},"1")

{'input': {'operation': 'SET', 'id': '1', 'default-cost': 1, 'vtn-path-cost': [{'port-desc': 'openflow:3,1,s3-eth1', 'cost': '100'}, {'port-desc': 'openflow:4,2,s4-eth2', 'cost': '1000'}]}}


{'output': {'status': 'CREATED'}}

In [21]:
odl.get_path_maps()

{'global-path-maps': {}}